In [1]:
import numpy as np
import os
import itertools
np.random.seed(113) #set seed before any keras import
import argparse
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, Dropout
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# logging
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# load dataset
with open('data/us.text') as f: 
    X_es = f.readlines()
with open('data/us.labels') as f: 
    Y_es = f.readlines()
    
assert len(X_es) == len(Y_es)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_es, Y_es, test_size=0.4, random_state=0)

In [5]:
logging.info(X_es[0])
logging.info(Y_es[0])

INFO:root:LoL @ West Covina, California 

INFO:root:2



In [6]:
# label encoding
label2idx = {label: i for i, label in enumerate(set(y_train))}
num_labels = len(label2idx.keys())
y_train = np_utils.to_categorical([label2idx[label] for label in y_train], num_classes=num_labels)
y_test = np_utils.to_categorical([label2idx[label] for label in y_test], num_classes=num_labels)

In [7]:
len(set(Y_es))

20